In [1]:
from prj.config import DATA_DIR
from prj.data.data_loader import DataConfig, DataLoader
import polars as pl
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor
from prj.data.data_loader import PARTITIONS_DATE_INFO
import pandas as pd
import gc

2024-12-28 10:32:47.549048: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-28 10:32:47.549082: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-28 10:32:47.550398: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-28 10:32:47.557283: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-28 10:32:48.353723: W tensorflow/compiler/tf2

In [2]:
def evaluate_model(model, train_ds: pl.LazyFrame, val_ds: pl.LazyFrame, features, cat_features=[], target='responder_6'):
    if len(cat_features) > 0:
        print(f'Using categorical features: {cat_features}')
    
    X = train_ds.select(features).collect().to_numpy()
    # X[cat_features] = X[cat_features].astype('category')
    y = train_ds.select(target).collect().to_series().to_numpy()
    w = train_ds.select('weight').collect().to_series().to_numpy()
    
    print(f'Fitting model: {X.shape}')
    model.fit(X, y, sample_weight=w, verbose=100, cat_features=cat_features)
    
    del X, y, w
    gc.collect()
    
    X_val = val_ds.select(features).collect().to_numpy()
    # X_val[cat_features] = X_val[cat_features].astype('category')
    y_val = val_ds.select(target).collect().to_series().to_numpy()
    w_val = val_ds.select('weight').collect().to_series().to_numpy()
    print(f'Evaluating model: {X_val.shape}')

    y_hat_val = model.predict(X_val).clip(-5, 5)
    return r2_score(y_val, y_hat_val, sample_weight=w_val)

In [3]:
# start_dt, end_dt = 1100, 1150
# start_val_dt = 1130

start_dt, end_dt = PARTITIONS_DATE_INFO[8]['min_date'], PARTITIONS_DATE_INFO[9]['max_date']
start_val_dt = PARTITIONS_DATE_INFO[9]['min_date']

In [4]:
params = {'iterations': 600, 'learning_rate': 0.05, 'task_type': 'GPU', 'has_time': False}

In [5]:
data_args = {}

config = DataConfig(**data_args)
loader = DataLoader(data_dir=DATA_DIR, config=config)
complete_ds = loader.load(start_dt, end_dt)
features = loader.features
cat_features = []


train_ds = complete_ds.filter(pl.col('date_id') < start_val_dt)
val_ds = complete_ds.filter(pl.col('date_id') >= start_val_dt)


model = CatBoostRegressor(**params)

evaluate_model(model, train_ds, val_ds, features, cat_features=cat_features)

Fitting model: (6140024, 79)
0:	learn: 0.8299093	total: 155ms	remaining: 1m 32s
100:	learn: 0.8200812	total: 14s	remaining: 1m 9s
200:	learn: 0.8163687	total: 28.2s	remaining: 55.9s
300:	learn: 0.8132383	total: 42.6s	remaining: 42.3s
400:	learn: 0.8102748	total: 57.2s	remaining: 28.4s
500:	learn: 0.8080570	total: 1m 11s	remaining: 14.2s
599:	learn: 0.8058844	total: 1m 25s	remaining: 0us
Evaluating model: (6274576, 79)


0.006223976325912539

In [6]:
data_args = {'include_time_id': True}

config = DataConfig(**data_args)
loader = DataLoader(data_dir=DATA_DIR, config=config)
complete_ds = loader.load(start_dt, end_dt)
features = loader.features
cat_features = []


train_ds = complete_ds.filter(pl.col('date_id') < start_val_dt)
val_ds = complete_ds.filter(pl.col('date_id') >= start_val_dt)


model = CatBoostRegressor(**params)

evaluate_model(model, train_ds, val_ds, features, cat_features=cat_features)

Fitting model: (6140024, 80)
0:	learn: 0.8299093	total: 160ms	remaining: 1m 35s
100:	learn: 0.8197053	total: 15.2s	remaining: 1m 15s
200:	learn: 0.8156525	total: 30.4s	remaining: 1m
300:	learn: 0.8125707	total: 45.8s	remaining: 45.5s
400:	learn: 0.8093001	total: 1m 1s	remaining: 30.5s
500:	learn: 0.8069392	total: 1m 17s	remaining: 15.2s
599:	learn: 0.8046996	total: 1m 32s	remaining: 0us
Evaluating model: (6274576, 80)


0.006580993598250595

In [7]:
data_args = {'include_time_id': True, 'include_intrastock_norm_temporal': True}

config = DataConfig(**data_args)
loader = DataLoader(data_dir=DATA_DIR, config=config)
complete_ds = loader.load(start_dt, end_dt)
features = loader.features
cat_features = []


train_ds = complete_ds.filter(pl.col('date_id') < start_val_dt)
val_ds = complete_ds.filter(pl.col('date_id') >= start_val_dt)


model = CatBoostRegressor(**params)

evaluate_model(model, train_ds, val_ds, features, cat_features=cat_features)

100%|██████████| 346/346 [00:21<00:00, 16.34it/s]


Fitting model: (6140024, 134)
0:	learn: 0.8298951	total: 197ms	remaining: 1m 57s
100:	learn: 0.8189474	total: 19.1s	remaining: 1m 34s
200:	learn: 0.8145956	total: 37.9s	remaining: 1m 15s
300:	learn: 0.8113661	total: 56.5s	remaining: 56.2s
400:	learn: 0.8083602	total: 1m 15s	remaining: 37.3s
500:	learn: 0.8058360	total: 1m 33s	remaining: 18.6s
599:	learn: 0.8032523	total: 1m 52s	remaining: 0us
Evaluating model: (6274576, 134)


0.006761950067025735

In [8]:
pd.DataFrame(model.get_feature_importance(), index=features, columns=['importance']).sort_values(by='importance', ascending=False).style.background_gradient(cmap='viridis')

,importance
time_id_norm,6.047462
feature_08,3.979471
feature_04,3.297494
feature_36,3.215399
feature_38_responder_6_mean,2.882543
feature_61,2.619842
feature_75,2.555603
feature_05_responder_6_mean,2.466924
feature_58,2.399103
feature_01,2.278528
